# Тестируем облачную функцию

In [ ]:
import requests
import base64
import json
from index import handler
from gpx_utils import get_yandex_maps_json, config


config['local_cache_dir'] = '../'

Общие параметры

In [ ]:
# Пример карты из Конструктора карт
map_url = r'https://www.yandex.ru/maps/?um=constructor%3A971ee70a42c0ded3a2bd90b4260df279fa36cdcd6d87473250011f3137a28970'
# Пример карты с маршрутом навигации
# map_url = r'https://yandex.ru/maps/-/CHvPJI4z'
# Пример карты с двумя вариантами маршрута навигации (в gpx сохраняется выбранный/активный вариант)
# map_url = r'https://yandex.ru/maps/-/CHvPJQyY'
# Примеры одной и той же карты с выбором разных вариантов маршрута навигации (в gpx сохраняется выбранный вариант)
# map_url = r'https://yandex.ru/maps/-/CHCIVIZh'
# map_url = r'https://yandex.ru/maps/-/CHCI6R2F'
# Пример карты с рисованием маршрута линейкой
# map_url = r'https://yandex.ru/maps/-/CHCXmIl9'
# Пример карты с длинным маршрутом навигации routePoints - НЕ ПОДДЕРЖИВАЕТСЯ - нет координат маршрута
# map_url = r'https://yandex.ru/maps/-/CHGUyM2M'
# map_url = r'https://yandex.ru/maps/-/CHX0FSMg'
# Пример карты с 1 вариантом маршрута навигации (проблема индекса) - поддержка добавлена
# map_url = r'https://yandex.ru/maps/-/CHXzBFmJ'
# Сложная пользовательская карта (перегрузка X-GPX-Info, если не лимитировать выдачу)
# map_url = r'https://yandex.ru/maps/-/CHhdaY2M'
# Ошибка Карт - Не удалось проложить маршрут
# map_url = r'https://yandex.ru/maps/-/CLaN5K38'

params = {'url': map_url,
          'lines': 'tracks', #'routes', 'tracks' or 'track_segments'
          'placemarks': '1',
          'add_elevation': '1'}

### 1. Тестируем онлайн в облаке

In [ ]:
# URL функции в файле в результате работы deploy.sh
with open('func-url.cfg', 'r') as file:
    func_url = file.read()
    
try:
    # Отправляем GET-запрос
    response = requests.get(func_url, params=params, headers={'User-Agent': ''})
    if response.status_code != 200:
        print(f"Функция вернула ошибку: HTTP {response.status_code}")
    print(response.text)
    # В случае успеха выводится содержимое gpx: <?xml version="1.0" encoding="UTF-8"?>...
        
except Exception as e:
    print(f"Произошла ошибка: {e}")

In [ ]:
print(json.dumps(dict(response.headers)))

In [ ]:
print(base64.b64decode(response.headers['X-GPX-Info']).decode('utf-8'))

### 2. Локальные тесты

Общие параметры

In [ ]:
params_local = {
    'queryStringParameters': params,
    'headers': {
        'User-Agent': 'Mozilla/5.0'
        }
    }

#### 2.1 Тестируем основной вариант

In [ ]:
# В случае успеха выводится содержимое gpx: <?xml version="1.0" encoding="UTF-8"?>...
response = handler(params_local, None)
print(response['body'])

In [ ]:
print(response['headers'])

In [ ]:
print(base64.b64decode(response['headers']['X-GPX-Info']).decode('utf-8'))

#### 2.2. Тестируем локально с загрузкой данных клиентом

Гипотетический сценарий, который не будет работать в веб-приложении из-за CORS.

In [ ]:
params_local['body'] = get_yandex_maps_json(params_local['queryStringParameters'].pop('url'))

# В случае успеха выводится содержимое gpx: <?xml version="1.0" encoding="UTF-8"?>...
response = handler(params_local, None)
print(response['body'])

In [ ]:
print(response['headers'])